In [1]:
# from fastFM import als 
import pandas as pd
from numpy.linalg import norm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.metrics import mean_squared_error,recall_score,ndcg_score
from sklearn.model_selection import ShuffleSplit
from scipy.sparse import csc_matrix

'''
https://github.com/ibayer/fastFM
'''

'\nhttps://github.com/ibayer/fastFM\n'

In [2]:
import pywFM
import numpy as np
import pandas as pd

In [81]:
rs = ShuffleSplit(n_splits=5, test_size=.1, random_state=42)
for i, (train_index, test_index) in enumerate(rs.split(X)):
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]

In [3]:
movie_genre = np.load('../hw3_data/Movielens/movie_genre.npy')
movie_movie = np.load('../hw3_data/Movielens/movie_movie.npy')
user_age = np.load('../hw3_data/Movielens/user_age.npy')
user_movie = np.load('../hw3_data/Movielens/user_movie.npy')
user_occupation = np.load('../hw3_data/Movielens/user_occupation.npy')
user_user = np.load('../hw3_data/Movielens/user_user.npy')

X = user_movie[:,:2]
y = user_movie[:,-1]
entity_user = 943
entity_item = 1682
age, occupation, genre = 8, 21, 18 # entity amount 

In [17]:
def get_embedding(data, ):
    user, item = data
    user_index = np.zeros((1, entity_user), dtype=int)
    user_index[0,user-1] = 1

    item_index = np.zeros((1, entity_item), dtype=int)
    item_index[0, item-1] = 1

    fea_age = np.zeros((1, age), dtype=int)
    ua = user_age[user_age[:,0] == user][:,1]
    fea_age[0, ua-1] = 1

    fea_occupation = np.zeros((1, occupation), dtype=int)
    uo = user_occupation[user_occupation[:,0] == user][:,1]
    fea_occupation[0, uo-1] = 1

    fea_genre = np.zeros((1, genre), dtype=int)
    mg = movie_genre[movie_genre[:,0] == item][:,1]
    fea_genre[0, mg-1] = 1

    user_knn = np.zeros((1, entity_user), dtype=int)
    uu = user_user[user_user[:,0] == user][:,1].astype(int)
    user_knn[0, uu-1] = 1

    item_knn = np.zeros((1, entity_item), dtype=int)
    mm = movie_movie[movie_movie[:,0] == item][:,1].astype(int)
    item_knn[0, mm-1] = 1
    
    train_example = np.concatenate((user_index, item_index, fea_age, fea_occupation, fea_genre, user_knn, item_knn), axis=1)

    return train_example 


embed_X = np.apply_along_axis(get_embedding, 1, X)
# np.save("../hw3_data/Movielens/embedding.npy",embed_X)

(1, 5297)

In [4]:
embed_X = np.load("../hw3_data/Movielens/embedding.npy")

In [32]:
embed_X.shape

(99543, 5297)

In [74]:
user_index, item_index, fea_age, fea_occupation, fea_genre, user_knn, item_knn

(627, 649)

In [33]:
fm = pywFM.FM(task='regression', num_iter=50)
model = fm.run(embed_X[:80000], y[:80000], embed_X[80000:], target[80000:])
print(model.predictions)
# you can also get the model weights
print(model.weights)


OSError: `LIBFM_PATH` is not set. Please install libFM and set the path variable (https://github.com/jfloff/pywFM#installing).